## Compute frequencies for subset

In [ ]:
import pandas as pd
from tqdm import tqdm
import errant
import spacy
import seaborn as sns
import matplotlib.pyplot as plt

nlp = spacy.load('en_core_web_sm')
annotator = errant.load('en', nlp)

df = pd.read_csv("./data/corrected_data.csv")

error_types = []
temp_data = df.head(5)
for i in tqdm(range(len(df))):
    temp_data = df.iloc[i]
    orig = annotator.parse(temp_data['original'])
    cor = annotator.parse(temp_data['modified'])
    edits = annotator.annotate(orig, cor)
    for e in edits:
        error_types.append(e.type)
        #print(e.o_start, e.o_end, e.o_str, e.c_start, e.c_end, e.c_str, e.type)
        
error_df = pd.DataFrame(columns = ['type'], data=error_types)
#error_df.head()

#error_df.value_counts(normalize=True)

freq = pd.DataFrame(error_df.value_counts(), columns=['count'])
freq = freq.reset_index()

plt.figure(figsize=(20,8))
pl = sns.barplot(x='type', y='count', data=freq)
plt.xticks(rotation=45)

pl.axes.set_yscale('log')
plt.show()

## Generate more data

In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import spacy 
import errant

parq = pd.read_parquet('./data/0001.parquet', engine='fastparquet')
df = parq.head(50000)

nlp = spacy.load('en_core_web_sm')
annotator = errant.load('en', nlp)

for i in tqdm(range(len(df))):
    temp_data = df.iloc[i]
    orig = annotator.parse(temp_data['input'])
    cor = annotator.parse(temp_data['output'])
    edits = annotator.annotate(orig, cor)
    df.loc[i, 'errors'] = str([e.type for e in edits]).replace('[', '').replace(']','').replace("'",'')
    #print(e.o_start, e.o_end, e.o_str, e.c_start, e.c_end, e.c_str, e.type)
        
df.to_csv("./data/errors50k.csv")

  0%|          | 0/50000 [00:00<?, ?it/s]/var/folders/pl/wj42m65s2ng3tpsb296dtstc0000gn/T/ipykernel_21819/3126753873.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[i, 'errors'] = str([e.type for e in edits]).replace('[', '').replace(']','').replace("'",'')
100%|██████████| 50000/50000 [08:20<00:00, 99.85it/s] 
